# Set Membership

The cell below defines two **abstract classes**: the first represents a set and basic insert/search operations on it. You will need to impement this API four times, to implement (1) sequential search, (2) binary search tree, (3) balanced search tree, and (4) bloom filter. The second defines the synthetic data generator you will need to implement as part of your experimental framework. <br><br>**Do NOT modify the next cell** - use the dedicated cells further below for your implementation instead. <br>

In [2]:
# DO NOT MODIFY THIS CELL

from abc import ABC, abstractmethod  

# abstract class to represent a set and its insert/search operations
class AbstractSet(ABC):
    
    # constructor
    @abstractmethod
    def __init__(self):
        pass           
        
    # inserts "element" in the set
    # returns "True" after successful insertion, "False" if the element is already in the set
    # element : str
    # inserted : bool
    @abstractmethod
    def insertElement(self, element):     
        inserted = False
        return inserted   
    
    # checks whether "element" is in the set
    # returns "True" if it is, "False" otherwise
    # element : str
    # found : bool
    @abstractmethod
    def searchElement(self, element):
        found = False
        return found    
    
    
    
# abstract class to represent a synthetic data generator
class AbstractTestDataGenerator(ABC):
    
    # constructor
    @abstractmethod
    def __init__(self):
        pass           
        
    # creates and returns a list of length "size" of strings
    # size : int
    # data : list<str>
    @abstractmethod
    def generateData(self, size):     
        data = [""]*size
        return data

Use the cell below to define any auxiliary data structure and python function you may need. Leave the implementation of the main API to the next code cells instead.

In [3]:
# ADD AUXILIARY DATA STRUCTURE DEFINITIONS AND HELPER CODE HERE



Use the cell below to implement the requested API by means of **sequential search**.

In [4]:
class SequentialSearchSet(AbstractSet):
    
    def __init__(self):
        # ADD YOUR CODE HERE
        self.data = []

    def insertElement(self, element):
        # ADD YOUR CODE HERE
        if element not in self.data:
            self.data.append(element)
            return True
        return False

    def searchElement(self, element):
        # ADD YOUR CODE HERE
        for item in self.data:
            if item == element:
                return True
        return False

Use the cell below to implement the requested API by means of **binary search tree**.

In [5]:
class Node:
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None

class BinarySearchTreeSet(AbstractSet):
    
    def __init__(self):
        # ADD YOUR CODE HERE
        self.root = None

    def insertElement(self, element):
        # ADD YOUR CODE HERE
        if self.root is None:
            self.root = Node(element)
            return True
        current = self.root
        while current:
            if current.value == element:
                return False
            elif current.value > element:
                if current.left is None:
                    current.left = Node(element)
                    return True
                current = current.left
            else:
                if current.right is None:
                    current.right = Node(element)
                    return True
                current = current.right

    def searchElement(self, element):
        found = False
        # ADD YOUR CODE HERE
        current = self.root
        while current:
            if current.value == element:
                found = True
                break
            elif current.value > element:
                current = current.left
            else:
                current = current.right
        return found

Use the cell below to implement the requested API by means of **balanced search tree**.

In [6]:
class RBNode:
    def __init__(self, value, parent, color=False):
        self.value = value
        self.left = None
        self.right = None
        self.color = color
        self.parent = parent


class BalancedSearchTreeSet(AbstractSet):
    def __init__(self):
        self.root = None

    def searchElement(self, element):
        found = False
        current = self.root
        while current:
            if current.value == element:
                found = True
                break
            elif current.value > element:
                current = current.left
            else:
                current = current.right
        return found

    def insertElement(self, element):
        if self.root is None:
            self.root = RBNode(element, None, False)
            return True
        current = self.root
        while current:
            if current.value == element:
                return False
            elif current.value > element:
                if current.left is None:
                    current.left = RBNode(element, current, True)
                    self.balanceTree(current.left)
                    break
                current = current.left
            else:
                if current.right is None:
                    current.right = RBNode(element, current, True)
                    self.balanceTree(current.right)
                    break
                current = current.right
        return True

    def balanceTree(self, new_node):
        while new_node != self.root and new_node.parent.color:
            if new_node.parent == new_node.parent.parent.right:
                uncle = new_node.parent.parent.left
                if uncle is None:
                    return
                if uncle.color:
                    self.flipColors(new_node.parent.parent)
                else:
                    if new_node == new_node.parent.left:
                        new_node = new_node.parent
                        self.rotateRight(new_node)
                    new_node.parent.color = False
                    new_node.parent.parent.color = True
                    self.rotateLeft(new_node.parent.parent)
            else:
                uncle = new_node.parent.parent.right  # uncle
                if uncle is None:
                    return
                if uncle.color:
                    self.flipColors(new_node.parent.parent)
                else:
                    if new_node == new_node.parent.right:
                        new_node = new_node.parent
                        self.rotateLeft(new_node)
                    new_node.parent.color = False
                    new_node.parent.parent.color = True
                    self.rotateRight(new_node.parent.parent)
        self.root.color = False

    def rotateLeft(self, node):
        x = node.right
        node.right = x.left
        x.left = node
        x.color = node.color
        node.color = True
        return x

    def rotateRight(self, node):
        x = node.left
        node.left = x.right
        x.right = node
        x.color = node.color
        node.color = True
        return x

    def flipColors(self, node):
        node.color = True
        node.left.color = False
        node.right.color = False

    def isRed(self, node):
        if node is None:
            return False
        return node.color


Use the cell below to implement the requested API by means of **bloom filter**.

In [19]:
import bitarray

class BloomFilterSet(AbstractSet):
    def __init__(self):
        # ADD YOUR CODE HERE
        self.size = 200
        self.num_hashes = 4
        self.bit_array = bitarray.bitarray(self.size)
        self.bit_array.setall(0)

    # FNV-1a hash
    def hash(self, item, seed):
        fnv_offset_basis = 0x811c9dc5
        fnv_prime = 0x01000193
        hash_val = fnv_offset_basis ^ seed
        for char in item:
            hash_val = (hash_val ^ ord(char)) * fnv_prime
        return hash_val % self.size

    # Simple hash
    def _hash(self, item, seed):
        hash_val = 0
        for char in item:
            hash_val = (hash_val * seed + ord(char)) % self.size
        return hash_val

    def insertElement(self, item):
        inserted = False
        for seed in range(self.num_hashes):
            hash_val = self.hash(item, seed)
            if self.bit_array[hash_val] == 0:
                inserted = True
            self.bit_array[hash_val] = 1
        return inserted

    def searchElement(self, item):
        for seed in range(self.num_hashes):
            hash_val = self.hash(item, seed)
            if self.bit_array[hash_val] == 0:
                return False
        return True

bloom = BloomFilterSet()
bloom.insertElement("hello")
bloom.insertElement("world")
print(bloom.insertElement("bye"))
print(bloom.insertElement("bye"))
print(bloom.searchElement("hello"))
print(bloom.searchElement("world"))
print(bloom.searchElement("hello world"))

True
False
True
True
False


Use the cell below to implement the **synthetic data generator** as part of your experimental framework.

In [8]:
import string
import random

class TestDataGenerator(AbstractTestDataGenerator):
    
    def __init__(self):
        # ADD YOUR CODE HERE
        
        pass           
        
    def generateData(self, size):     
        # ADD YOUR CODE HERE
        data = [""]*size
        

        return data   



Use the cells below for the python code needed to **fully evaluate your implementations**, first on real data and subsequently on synthetic data (i.e., read data from test files / generate synthetic one, instantiate each of the 4 set implementations in turn, then thorouhgly experiment with insert/search operations and measure their performance).

In [ ]:
import timeit

# ADD YOUR TEST CODE HERE TO WORK ON REAL DATA
files = ["test1-mobydick.txt", "test2-warpeace.txt", "test3-dickens.txt"]
with open("testfiles/test-search.txt", "r") as t:
    search_data = t.read().split()

def test_real_data():
    for file in files:
        with open(f"testfiles/{file}", "r") as f:
            data = f.read().split()
            bst = BalancedSearchTreeSet()
            insert_time = timeit.timeit(lambda: [bst.insertElement(word) for word in data], number=1)
            search_time = timeit.timeit(lambda: [bst.searchElement(search_word) for search_word in search_data], number=1)
            print(f"{file[6:-4]}, LLRBTree insert time: {insert_time}")
            print(f"{file[6:-4]}, LLRBTree search time:  {search_time}")

test_real_data()

warpeace, LLRBTree insert time: 6.809470540843904
warpeace, LLRBTree search time:  0.026664209086447954


KeyboardInterrupt: 

In [ ]:
import timeit

# ADD YOUR TEST CODE HERE TO WORK ON SYNTHETIC DATA





Testing the correctness of searches and error of bloom filter

In [21]:
def test_bloom_effectiveness():
    bloom = BloomFilterSet()
    LLRB = BinarySearchTreeSet()
    for file in files:
        with open(f"testfiles/{file}", "r") as f:
            data = f.read().split()
            for word in data:
                LLRB.insertElement(word)
                bloom.insertElement(word)
            false = 0
            for word in search_data:
                if bloom.searchElement(word) and not LLRB.searchElement(word):
                    false += 1
            print(f"{file}, false positives: {false}, probability: {round(false/len(data), 4)}")

def test_search_correctness():
    bst = BinarySearchTreeSet
    LLRB = BalancedSearchTreeSet
    for file in files:
        with open(f"testfiles/{file}", "r") as f:
            data = f.read().split()
            for word in data:
                bst.insertElement(word)
                LLRB.insertElement(word)
            for word in data:
                if not bst.searchElement(word) or not LLRB.searchElement(word):
                    raise Exception("Search not working")


test_bloom_effectiveness()

test1-mobydick.txt, false positives: 69, probability: 0.0003
test2-warpeace.txt, false positives: 39, probability: 0.0001
test3-dickens.txt, false positives: 25, probability: 0.0
